# Run Agent
- Implement a workflow to write a daily AI newsletter
- see README.md for details


In [1]:
# run end to end
# fix hdbscan with existing db somehow


In [2]:
import os
import yaml
import dotenv
import logging
import json
import yaml
from datetime import datetime
import time
import random
import glob
import pickle
import sqlite3

from pathlib import Path

import asyncio
import nest_asyncio

import pydantic
from pydantic import BaseModel, Field, RootModel
from typing import Dict, TypedDict, Type, List, Optional, Any, Iterable, Text
from dataclasses import dataclass, field
from enum import Enum

import numpy as np
import pandas as pd

import pandas as pd
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import hdbscan

import openai
from openai import AsyncOpenAI

import agents
from agents.exceptions import InputGuardrailTripwireTriggered
from agents import (Agent, Runner, Tool, OpenAIResponsesModel, 
                    ModelSettings, FunctionTool, InputGuardrail, GuardrailFunctionOutput,
                    SQLiteSession, set_default_openai_api, set_default_openai_client
                   )


import tenacity
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

from IPython.display import HTML, Image, Markdown, display

from log_handler import SQLiteLogHandler, setup_sqlite_logging, sanitize_error_for_logging
from config import LOGDB
from llm import LLMagent, LangfuseClient  # methods to apply prompts async to large batches
from db import Url 

from fetch import Fetcher # fetch news urls
from newsletter_state import NewsletterAgentState, StepStatus
from news_agent import NewsletterAgent


In [3]:
print(f"OpenAI:            {openai.__version__}")
print(f"OpenAI Agents SDK  {agents.__version__}")
print(f"Pydantic           {pydantic.__version__}")


OpenAI:            1.109.0
OpenAI Agents SDK  0.3.1
Pydantic           2.11.9


In [4]:
dotenv.load_dotenv()

# to run async in jupyter notebook
nest_asyncio.apply()

# verbose OpenAI console logging if something doesn't work
# logging.basicConfig(level=logging.DEBUG)
# openai_logger = logging.getLogger("openai")
# openai_logger.setLevel(logging.DEBUG)


In [5]:
# modules create a default logger, or we can pass this logger

def setup_logging(session_id: str = "default", db_path: str = "agent_logs.db") -> logging.Logger:
    """Set up logging to console and SQLite database."""

    # Create logger
    logging.basicConfig(level=logging.INFO)

    logger = logging.getLogger(f"NewsletterAgent.{session_id}")
    logger.setLevel(logging.INFO)

    # Clear any existing handlers
    logger.handlers.clear()

    # Console handler
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    console_formatter = logging.Formatter(
        '%(asctime)s | %(name)s | %(levelname)s | %(message)s',
        datefmt='%H:%M:%S'
    )
    console_handler.setFormatter(console_formatter)

    # SQLite handler
    sqlite_handler = SQLiteLogHandler(db_path)
    sqlite_handler.setLevel(logging.INFO)
    sqlite_formatter = logging.Formatter('%(message)s')
    sqlite_handler.setFormatter(sqlite_formatter)

    # Add handlers to logger
    logger.addHandler(console_handler)
    logger.addHandler(sqlite_handler)

    # Prevent propagation to root logger
    logger.propagate = False

    return logger

logger = setup_logging("newsletter_agent", "test_logs.db")

# Log some test messages
logger.info("Test info message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.warning("Test warning message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.error("Test error message", extra={
    'step_name': 'error_step',
    'agent_session': 'demo_session'
})

sanitize_error_for_logging("log with some bad stuff for the filter: sk-proj-123456789012345678901234567890123456789012345678")

12:13:20 | NewsletterAgent.newsletter_agent | INFO | Test info message
12:13:20 | NewsletterAgent.newsletter_agent | WARNING | Test warning message
12:13:20 | NewsletterAgent.newsletter_agent | ERROR | Test error message


'log with some bad stuff for the filter: [API_KEY_REDACTED]'

# Run Agent Worfklow

In [8]:
print("🚀 Creating NewsletterAgent...")

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set")

# Set up OpenAI client for the agents SDK
set_default_openai_client(AsyncOpenAI(api_key=api_key))
try:
    # set up state
    session_id = 'test_newsletter_20251006080143257231'
    step_name = 'step_06_cluster_by_topic'
#     del session_id, step_name
except Exception as e:
    print(e)

do_download=False
process_since=None
# process_since='2025-10-05 18:30:00'

# Create agent with persistent state
if 'session_id' in vars():
    # load state from db for session_id and state
    print("session_id is defined")
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, 
                                 db_path="newsletter_agent.db", 
                                 do_download=do_download,
                                 process_since=process_since,
                                 verbose=False
                                )
    state = state.load_from_db(step_name)
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=True, timeout=30)    
else:
    # create new session
    print("session_id is not defined")
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S%f")    
    session_id = f"test_newsletter_{timestamp}"
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, 
                                 db_path="newsletter_agent.db",
                                 do_download=do_download,
                                 process_since=process_since,
                                 verbose=False
                                ) 
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=False, timeout=30)
    state.serialize_to_db("initialize")

12:15:58 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Using provided state with 83 articles


🚀 Creating NewsletterAgent...
session_id is defined
test_newsletter_20251006080143257231
Initialized NewsletterAgent with persistent state and 9-step workflow
Session ID: test_newsletter_20251006080143257231


In [9]:
agent.state.get_status()


{'headlines': {'total': 83},
 'sources': {'config_file': 'sources.yaml', 'loaded_sources': 0},
 'topics': {'cluster_topics': 0, 'topics': []},
 'workflow': {'current_step': 'step_07_select_sections',
  'workflow_complete': False,
  'workflow_status': 'started',
  'workflow_status_message': '',
  'progress_percentage': 66.66666666666666,
  'max_edits': 2,
  'concurrency': 16,
  'do_download': False,
  'process_since': datetime.datetime(2025, 10, 5, 18, 30)},
 'processing': {'topic_clusters': 0,
  'newsletter_sections': 0,
  'final_newsletter_length': 0}}

In [16]:
# User prompt to run workflow
user_prompt = "Show the workflow status"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


📝 User prompt: 'Show the workflow status'


08:01:46 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Starting check_workflow_status
08:01:46 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Completed check_workflow_status


⏱️  Total execution time: 6.95s
📊 Final result:
Current workflow status:
- Progress: 0.0% (0/9 complete)
- Status: 0 complete, 0 started, 0 failed, 9 not started
- Next step: Step 1 — Fetch URLs

Step details:
- Step 1: Fetch Urls — not_started
- Step 2: Filter Urls — not_started
- Step 3: Download Articles — not_started
- Step 4: Extract Summaries — not_started
- Step 5: Rate Articles — not_started
- Step 6: Cluster By Topic — not_started
- Step 7: Select Sections — not_started
- Step 8: Draft Sections — not_started
- Step 9: Finalize Newsletter — not_started

What would you like me to do next? Options: run all steps, run a specific step (1–9), or resume/continue.


In [17]:
# User prompt to run a workflow step
user_prompt = "Run step 1, fetch urls"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)



📝 User prompt: 'Run step 1, fetch urls'


08:01:55 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Starting check_workflow_status
08:01:55 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Completed check_workflow_status
08:01:56 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Starting Step 1: Gather URLs
2025-10-06 08:01:56,514 - fetcher_5760478608 - INFO - [fetcher_init] Loading sources from sources.yaml
2025-10-06 08:01:56,528 - fetcher_5760478608 - INFO - [fetcher_init] Loaded 17 sources: 7 RSS, 9 HTML, 1 API
2025-10-06 08:01:56,529 - fetcher_5760478608 - DEBUG - [fetcher_sources] Source 'Ars Technica': type=RSS, url=https://arstechnica.com/ai/
2025-10-06 08:01:56,529 - fetcher_5760478608 - DEBUG - [fetcher_sources] Source 'Bloomberg': type=RSS, url=https://www.bloomberg.com/ai
2025-10-06 08:01:56,530 - fetcher_5760478608 - DEBUG - [fetcher_sources] Source 'Business Insider': type=html, url=https://www.businessinsider.com/tech
2025-10-06 08:01:56,530 - fetcher_5760478608 - DEBU

2025-10-06 08:01:56,714 - fetcher_5760478608 - INFO - [fetch_html] Parsing HTML file: download/sources/VentureBeat.html


▶ Starting Step 1: step_01_fetch_urls


2025-10-06 08:01:56,720 - fetcher_5760478608 - INFO - [fetch_html] Parsed HTML file: download/sources/VentureBeat.html
2025-10-06 08:01:56,720 - fetcher_5760478608 - INFO - [fetch_html] HTML fetch successful for VentureBeat: 16 articles
2025-10-06 08:01:56,721 - fetcher_5760478608 - INFO - [fetch_html] Using existing HTML file from WSJ: https://www.wsj.com/tech/ai
2025-10-06 08:01:56,721 - fetcher_5760478608 - INFO - [fetch_html] Parsing HTML file: download/sources/WSJ.html
2025-10-06 08:01:56,743 - fetcher_5760478608 - INFO - [fetch_html] Parsed HTML file: download/sources/WSJ.html
2025-10-06 08:01:56,743 - fetcher_5760478608 - INFO - [fetch_html] HTML fetch successful for WSJ: 26 articles
2025-10-06 08:01:56,743 - fetcher_5760478608 - INFO - [fetch_html] Using existing HTML file from Washington Post: https://www.washingtonpost.com/technology/innovations/
2025-10-06 08:01:56,744 - fetcher_5760478608 - INFO - [fetch_html] Parsing HTML file: download/sources/Washington_Post.html
2025-10

,source,url
0,Ars Technica,20
1,Bloomberg,25
2,Business Insider,16
3,FT,48
4,Feedly AI,36
5,Hacker News,30
6,HackerNoon,50
7,New York Times,23
8,NewsAPI,24
9,Reddit,52


,source,title,url,published,rss_summary,id
0,Ars Technica,A biological 0-day? Threat-screening tools may...,https://arstechnica.com/science/2025/10/do-ai-...,"Fri, 03 Oct 2025 20:12:52 +0000",Ordering DNA for AI-designed toxins doesn't al...,0
1,Ars Technica,Ars Live: Is the AI bubble about to pop? A liv...,https://arstechnica.com/ai/2025/10/ars-live-is...,"Fri, 03 Oct 2025 17:49:26 +0000",Join a live discussion on October 7 about the ...,1
2,Ars Technica,Why iRobot’s founder won’t go within 10 feet o...,https://arstechnica.com/ai/2025/10/why-irobots...,"Thu, 02 Oct 2025 21:10:23 +0000",Rodney Brooks says humanoid robots pose hidden...,2
3,Ars Technica,Meta won’t allow users to opt out of targeted ...,https://arstechnica.com/tech-policy/2025/10/me...,"Thu, 02 Oct 2025 17:16:38 +0000",US users stuck with AI ad targeting as EU user...,3
4,Ars Technica,OpenAI mocks Musk’s math in suit over iPhone/C...,https://arstechnica.com/tech-policy/2025/10/op...,"Wed, 01 Oct 2025 21:15:09 +0000",xAI's claim that Apple gave ChatGPT a monopoly...,4
...,...,...,...,...,...,...
494,NewsAPI,Greenleaf Trust Trims Position in Booz Allen H...,https://www.etfdailynews.com/2025/10/05/greenl...,2025-10-05T08:12:55Z,NaN,494
495,NewsAPI,"Signaturefd LLC Acquires 1,594 Shares of Sound...",https://www.etfdailynews.com/2025/10/05/signat...,2025-10-05T08:10:49Z,NaN,495
496,NewsAPI,Can AI get shoppers discounts and save people ...,https://www.postandcourier.com/business/artifi...,2025-10-05T09:02:17Z,NaN,496
497,NewsAPI,Laid off? Here's why losing your job might be ...,https://www.foxnews.com/opinion/laid-off-heres...,2025-10-05T12:00:51Z,NaN,497


08:01:57 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Completed Step 1: Gathered 585 articles
08:01:58 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Starting check_workflow_status
08:01:59 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Completed check_workflow_status


⏱️  Total execution time: 9.50s
📊 Final result:
Step 1 (Fetch URLs) completed.

Summary:
- Collected 585 headlines from 17 sources (RSS).
- 499 articles stored in persistent state.
- Workflow progress: 1/9 steps complete (11.1%).
- Next step: Step 2 — Filter URLs

Would you like me to continue with Step 2 now?


In [18]:
pd.DataFrame(state.headline_data) 


,source,title,url,published,rss_summary,id
0,Ars Technica,A biological 0-day? Threat-screening tools may...,https://arstechnica.com/science/2025/10/do-ai-...,"Fri, 03 Oct 2025 20:12:52 +0000",Ordering DNA for AI-designed toxins doesn't al...,0
1,Ars Technica,Ars Live: Is the AI bubble about to pop? A liv...,https://arstechnica.com/ai/2025/10/ars-live-is...,"Fri, 03 Oct 2025 17:49:26 +0000",Join a live discussion on October 7 about the ...,1
2,Ars Technica,Why iRobot’s founder won’t go within 10 feet o...,https://arstechnica.com/ai/2025/10/why-irobots...,"Thu, 02 Oct 2025 21:10:23 +0000",Rodney Brooks says humanoid robots pose hidden...,2
3,Ars Technica,Meta won’t allow users to opt out of targeted ...,https://arstechnica.com/tech-policy/2025/10/me...,"Thu, 02 Oct 2025 17:16:38 +0000",US users stuck with AI ad targeting as EU user...,3
4,Ars Technica,OpenAI mocks Musk’s math in suit over iPhone/C...,https://arstechnica.com/tech-policy/2025/10/op...,"Wed, 01 Oct 2025 21:15:09 +0000",xAI's claim that Apple gave ChatGPT a monopoly...,4
...,...,...,...,...,...,...
494,NewsAPI,Greenleaf Trust Trims Position in Booz Allen H...,https://www.etfdailynews.com/2025/10/05/greenl...,2025-10-05T08:12:55Z,NaN,494
495,NewsAPI,"Signaturefd LLC Acquires 1,594 Shares of Sound...",https://www.etfdailynews.com/2025/10/05/signat...,2025-10-05T08:10:49Z,NaN,495
496,NewsAPI,Can AI get shoppers discounts and save people ...,https://www.postandcourier.com/business/artifi...,2025-10-05T09:02:17Z,NaN,496
497,NewsAPI,Laid off? Here's why losing your job might be ...,https://www.foxnews.com/opinion/laid-off-heres...,2025-10-05T12:00:51Z,NaN,497


In [19]:
countdf = pd.DataFrame(state.headline_data) \
    .groupby("source") \
    .count()[["id"]] \
    .reset_index() \
    .rename(columns={'id': 'count'}) \
    .sort_values("count", ascending=False)
countdf 


,source,count
9,Reddit,52
6,HackerNoon,50
12,The Register,50
3,FT,48
4,Feedly AI,36
5,Hacker News,30
16,Washington Post,29
13,The Verge,28
1,Bloomberg,25
8,NewsAPI,24


In [20]:
# Run tool directly without LLM processing an input prompt or results
# user_prompt = "Run step 2, filter urls"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("filter_urls")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


08:02:01 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Starting Step 2: Filter URLs
08:02:01 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | 🔍 Filtering 499 headlines...
08:02:01 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | 🔄 Checking for duplicates seen before 2025-10-05T18:30:00
08:02:01 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | 🔍 Filtering 499 articles for dupes.
08:02:01 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | URL deduplication with process_since: 298 URLs filtered (seen before 2025-10-05T18:30:00), 201 new URLs remain
08:02:01 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | 🔍 Filtering 499 headlines for AI relevance using LLM...
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/filter_urls' from Langfuse
INFO:llm:Parsed prompt 'newsagent/filter_urls': model=gpt-4.1-mini, system_len=459, user_len=954


▶ Starting Step 2: step_02_filter_urls
checking https://arstechnica.com/science/2025/10/do-ai-designed-proteins-create-a-biosecurity-vulnerability/
found before cutoff
checking https://arstechnica.com/ai/2025/10/ars-live-is-the-ai-bubble-about-to-pop-a-live-chat-with-ed-zitron/
found before cutoff
checking https://arstechnica.com/ai/2025/10/why-irobots-founder-wont-go-within-10-feet-of-todays-walking-robots/
found before cutoff
checking https://arstechnica.com/tech-policy/2025/10/meta-wont-allow-users-to-opt-out-of-targeted-ads-based-on-ai-chats/
found before cutoff
checking https://arstechnica.com/tech-policy/2025/10/openai-mocks-musks-math-in-suit-over-iphone-chatgpt-integration/
found before cutoff
checking https://arstechnica.com/ai/2025/10/openais-sora-2-lets-users-insert-themselves-into-ai-videos-with-sound/
found before cutoff
checking https://arstechnica.com/ai/2025/10/can-todays-ai-video-models-accurately-model-how-the-real-world-works/
found before cutoff
checking https://ars

08:02:16 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Completed Step 2: 90 AI-related articles, 298 duplicates removed


⏱️  Total execution time: 15.62s
📊 Final result:
✅ Step 2 step_02_filter_urls completed successfully! Removed 298 duplicate URLs, classified 201 new articles, found 90 AI-related.


In [21]:
# User prompt to run workflow
# user_prompt = "Run step 3, download full articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("download_articles")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

08:02:16 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Starting Step 3: Download Articles
08:02:16 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Starting concurrent scraping of 90 AI-related articles


▶ Starting Step 3: step_03_download_articles


08:02:16 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Launching browser for 90 URLs with 16 concurrent workers
08:02:16 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Worker 0 fetching 1 of 90 https://9to5mac.com/2025/10/06/when-people-create-sora-deepfakes-of-you-you-can-now-set-limits/
08:02:16 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | File already exists: download/html/When_people_create_Sora_deepfakes_of_you__you_can_now_set_limits.html
08:02:16 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Worker 0 completed https://9to5mac.com/2025/10/06/when-people-create-sora-deepfakes-of-you-you-can-now-set-limits/ with status: exists
08:02:16 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Worker 0 fetching 2 of 90 https://v.redd.it/06se9ohdudtf1
08:02:16 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | File already exists: download/html/I_m_definitely_falling_for_AI_when_I_m_older.html


08:02:16 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | File already exists: download/html/AI_drives_38__surge_in_global_venture_capital_funding_as_third_quarter_hits__97B.html
08:02:16 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Worker 0 completed https://siliconangle.com/2025/10/06/ai-drives-38-surge-global-venture-capital-funding-third-quarter-hits-97b/ with status: exists
08:02:16 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Worker 0 fetching 16 of 90 http://www.techmeme.com/251006/p2#a251006p2
08:02:16 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | File already exists: download/html/Melbourne-based_Heidi_Health__which_is_developing_AI_agents_to_assist_doctors__raised_a__65M_Series_B_led_by_Point72__taking_its_total_raised_to__96.6M__Dominic-Madori_Davis_TechCrunch.html
08:02:16 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Worker 0 completed http://www.techmeme.com/251006/p2#a251006p2 with s

08:02:16 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Worker 0 completed https://www.nytimes.com/2025/10/06/technology/openai-amd-chips.html with status: None
08:02:16 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Worker 0 fetching 29 of 90 https://www.cnbc.com/2025/10/05/openai-stargate-data-center-buildout-infrastructure-lead-keith-heyde.html
08:02:16 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | File already exists: download/html/This_Meta_alum_has_spent_10_months_leading_OpenAI_s_nationwide_hunt_for_its_Stargate_data_centers.html
08:02:16 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Worker 0 completed https://www.cnbc.com/2025/10/05/openai-stargate-data-center-buildout-infrastructure-lead-keith-heyde.html with status: exists
08:02:16 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Worker 0 fetching 30 of 90 https://www.bloomberg.com/news/articles/2025-10-06/ai-health-startup-heidi-gets-point72

...

08:02:17 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Worker 0 fetching 84 of 90 https://www.shortliffe.net/Buchanan-Shortliffe-1984/MYCIN%20Book.htm
08:02:17 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | File already exists: download/html/Rule-Based_Expert_Systems__The_Mycin_Experiments__1984.html
08:02:17 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Worker 0 completed https://www.shortliffe.net/Buchanan-Shortliffe-1984/MYCIN%20Book.htm with status: exists
08:02:17 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Worker 0 fetching 85 of 90 https://www.bloomberg.com/news/articles/2025-10-03/eliezer-yudkowsky-nate-soares-argue-ai-s-endgame-is-human-extinction
08:02:17 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Skipping ignored domain: www.bloomberg.com
08:02:17 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Worker 0 completed https://www.bloomberg.com/news/articles/2025-10-03/eli

08:02:17 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Worker 8 fetching 90 of 90 https://www.fastcompany.com/91413470/right-way-use-ai-work
08:02:17 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Rate limiting domain www.fastcompany.com, will retry later (need to wait 2.0s)
08:02:17 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Worker 8 re-queued rate-limited URL: https://www.fastcompany.com/91413470/right-way-use-ai-work
08:02:17 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Worker 9 fetching 90 of 90 https://www.fastcompany.com/91413470/right-way-use-ai-work
08:02:17 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Rate limiting domain www.fastcompany.com, will retry later (need to wait 2.0s)
08:02:17 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Worker 9 re-queued rate-limited URL: https://www.fastcompany.com/91413470/right-way-use-ai-work
08:02:17 | NewsletterAgent.test_newslett

Starting with 90 rows...
Processing 90 files...
Reading and truncating files to 8192 tokens using text-embedding-3-large tokenizer...
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Getting embeddings for 76 texts...


08:02:28 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Completed Step 3: Downloaded 76 articles


Creating indexed similarity matrix...
Finding pairs with similarity > 0.925...
Filtering dataframe...
  Pair: 41 vs 54
    41: Feedly AI - Stop Hallucinations at the Source: Hybrid RAG That Checks Itself
    54: HackerNoon - Stop Hallucinations at the Source: Hybrid RAG That Checks Itself
  Pair: 56 vs 42
    56: HackerNoon - The Data Infrastructure Behind Every Successful AI Startup
    42: Feedly AI - The Data Infrastructure Behind Every Successful AI Startup
  Pair: 43 vs 59
    43: Feedly AI - The GitHub Issue That Could Hijack Your AI Agent
    59: HackerNoon - The GitHub Issue That Could Hijack Your AI Agent
  Pair: 70 vs 39
    70: Techmeme - Sources: Oaktree, EQT, DWS, and other private capital groups are seeking to cash in on the US-driven AI boom, launching €17B of European data center sales (Financial Times)
    39: Feedly AI - Sources: Oaktree, EQT, DWS, and other private capital groups are seeking to cash in on the US-driven AI boom, launching €17B of European data center 

In [22]:
headline_df = state.headline_df
headline_df.loc[headline_df["status"]!=200]



,source,title,url,published,rss_summary,id,isAI,status,final_url,html_path,last_updated,text_path,content_length,domain,site_name,reputation
0,HackerNoon,The Complete Guide to Mega Productivity With ...,https://hackernoon.com/the-complete-guide-to-m...,"Mon, 06 Oct 2025 03:14:05 GMT",Perplexity’s Comet browser changed how I work—...,177,True,exists,https://hackernoon.com/the-complete-guide-to-m...,download/html/The_Complete_Guide_to_Mega_Produ...,None,download/text/The_Complete_Guide_to_Mega_Produ...,34068,hackernoon.com,Hacker Noon,2.0
1,HackerNoon,Stop Hallucinations at the Source: Hybrid RAG ...,https://hackernoon.com/stop-hallucinations-at-...,"Mon, 06 Oct 2025 03:27:48 GMT",Stop hallucinations. Validate every answer. Co...,176,True,exists,https://hackernoon.com/stop-hallucinations-at-...,download/html/Stop_Hallucinations_at_the_Sourc...,None,download/text/Stop_Hallucinations_at_the_Sourc...,31929,hackernoon.com,Hacker Noon,2.0
2,Hacker News,IRonCub: A Humanoid Robot Designed to Fly Like...,https://spectrum.ieee.org/ironcub-jet-powered-...,"Thu, 02 Oct 2025 18:54:12 +0000","<a href=""https://news.ycombinator.com/item?id=...",161,True,exists,https://spectrum.ieee.org/ironcub-jet-powered-...,download/html/IRonCub__A_Humanoid_Robot_Design...,None,download/text/IRonCub__A_Humanoid_Robot_Design...,18430,ieee.org,IEEE Xplore,1.0
3,Hacker News,The (economic) AI apocalypse is nigh,https://pluralistic.net/2025/09/27/econopocaly...,"Mon, 06 Oct 2025 10:51:53 +0000","<a href=""https://news.ycombinator.com/item?id=...",160,True,exists,https://pluralistic.net/2025/09/27/econopocaly...,download/html/The__economic__AI_apocalypse_is_...,None,download/text/The__economic__AI_apocalypse_is_...,18238,pluralistic.net,Pluralistic,0.0
4,Hacker News,Intro to BirdNET-Pi: Eavesdropping on my feath...,https://hannahilea.com/blog/birdnet-intro/,"Thu, 02 Oct 2025 14:49:12 +0000","<a href=""https://news.ycombinator.com/item?id=...",174,True,exists,https://hannahilea.com/blog/birdnet-intro/,download/html/Intro_to_BirdNET-Pi__Eavesdroppi...,None,download/text/Intro_to_BirdNET-Pi__Eavesdroppi...,16957,hannahilea.com,Hannah Ilea,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,Reddit,"‘If Anyone Builds It, Everyone Dies’ Is the Ne...",https://www.bloomberg.com/news/articles/2025-1...,NaN,NaN,290,True,skipped,https://www.bloomberg.com/news/articles/2025-1...,,None,,0,bloomberg.com,Bloomberg,5.0
82,Reddit,AI Slop Is Everywhere. What Happens Next?,https://www.wsj.com/tech/ai/ai-slop-is-everywh...,NaN,NaN,288,True,skipped,https://www.wsj.com/tech/ai/ai-slop-is-everywh...,,None,,0,wsj.com,The Wall Street Journal,5.0
83,New York Times,OpenAI Agrees to Use Computer Chips from AMD,https://www.nytimes.com/2025/10/06/technology/...,"Mon, 06 Oct 2025 11:22:50 +0000",Weeks after a $100 billion agreement to use co...,229,True,None,https://www.nytimes.com/2025/10/06/technology/...,,None,,0,nytimes.com,The New York Times,5.0
84,Feedly AI,"OpenAI, AMD Announce Massive Computing Deal, M...",https://www.wsj.com/tech/ai/openai-amd-announc...,NaN,NaN,120,True,skipped,https://www.wsj.com/tech/ai/openai-amd-announc...,,None,,0,wsj.com,The Wall Street Journal,5.0


In [23]:
headline_df = state.headline_df
headline_df.loc[headline_df["html_path"]==""]



,source,title,url,published,rss_summary,id,isAI,status,final_url,html_path,last_updated,text_path,content_length,domain,site_name,reputation
72,Bloomberg,AI Health Startup Heidi Gets Point72 Funds at ...,https://www.bloomberg.com/news/articles/2025-1...,NaN,NaN,25,True,skipped,https://www.bloomberg.com/news/articles/2025-1...,,None,,0,bloomberg.com,Bloomberg,5.0
73,WSJ,Elon Musk Gambles Billions in Memphis to Catch...,https://www.wsj.com/tech/elon-musk-xai-memphis...,NaN,NaN,411,True,skipped,https://www.wsj.com/tech/elon-musk-xai-memphis...,,None,,0,wsj.com,The Wall Street Journal,5.0
74,NewsAPI,7 Things Leaders Need to Know About Team AI Usage,https://bradenkelley.com/2025/10/7-things-lead...,2025-10-05T12:00:00Z,NaN,478,True,None,https://bradenkelley.com/2025/10/7-things-lead...,,None,,0,bradenkelley.com,Braden Kelley,0.0
75,Bloomberg,James Van Geelen on the Next Phase of the AI B...,https://www.bloomberg.com/news/articles/2025-1...,NaN,NaN,24,True,skipped,https://www.bloomberg.com/news/articles/2025-1...,,None,,0,bloomberg.com,Bloomberg,5.0
76,Hacker News,The G in GPU is for Graphics damnit,https://ut21.github.io/blog/triton.html,"Thu, 02 Oct 2025 11:46:27 +0000","<a href=""https://news.ycombinator.com/item?id=...",165,True,None,https://ut21.github.io/blog/triton.html,,None,,0,github.io,DeepFabric,0.0
77,Feedly AI,AI won’t just eliminate millions of jobs. It w...,https://www.fastcompany.com/91405505/ai-wont-j...,NaN,NaN,109,True,None,https://www.fastcompany.com/91405505/ai-wont-j...,,None,,0,fastcompany.com,Fast Company,2.0
78,Feedly AI,The right way to use AI at work,https://www.fastcompany.com/91413470/right-way...,NaN,NaN,110,True,None,https://www.fastcompany.com/91413470/right-way...,,None,,0,fastcompany.com,Fast Company,2.0
79,Bloomberg,OpenAI Announcements Make Waves in Stock Marke...,https://www.bloomberg.com/news/articles/2025-1...,NaN,NaN,23,True,skipped,https://www.bloomberg.com/news/articles/2025-1...,,None,,0,bloomberg.com,Bloomberg,5.0
80,Techmeme,AstraZeneca signs $555mn AI deal to develop ge...,http://www.techmeme.com/251006/p15#a251006p15,"Mon, 06 Oct 2025 08:00:45 -0400","<a href=""https://t.co/nn6cRhGN5w""><img align=""...",300,True,None,http://www.techmeme.com/251006/p15#a251006p15,,None,,0,techmeme.com,Techmeme,2.0
81,Reddit,"‘If Anyone Builds It, Everyone Dies’ Is the Ne...",https://www.bloomberg.com/news/articles/2025-1...,NaN,NaN,290,True,skipped,https://www.bloomberg.com/news/articles/2025-1...,,None,,0,bloomberg.com,Bloomberg,5.0


In [24]:
# User prompt to run workflow
# user_prompt = "Run step 4, Summarize articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("extract_summaries")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

08:02:49 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Processing 86 AI articles for summarization
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/extract_summaries' from Langfuse
INFO:llm:Parsed prompt 'newsagent/extract_summaries': model=gpt-4.1-mini, system_len=1273, user_len=43
08:02:49 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Using model 'gpt-4.1-mini' for summarization
08:02:49 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Starting summarization for 86 articles


▶ Starting Step 4: step_04_extract_summaries


08:02:54 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Extracting metadata from HTML files for 86 articles


⏱️  Total execution time: 7.08s
📊 Final result:
✅ Step 4 step_04_extract_summaries completed successfully! Generated AI-powered summaries for 86/86 articles.
💾 Summaries stored in headline DataFrame.


In [25]:
# if we get a refusal, examine prompt and delete rows
bad_stuff =["What\'s the Best Way ",
            "AI could make it eas",
            "AI can design toxic ",
           ]
print([len(s) for s in bad_stuff])
headline_df = state.headline_df
headline_df.loc[(headline_df["title"].str[:20].isin(bad_stuff))]



[20, 20, 20]


,source,title,url,published,rss_summary,id,isAI,status,final_url,html_path,last_updated,text_path,content_length,domain,site_name,reputation,summary,description,tags


In [26]:
# remove
state.headline_df_to_dict(headline_df.loc[~headline_df["title"].str[:20].isin(bad_stuff)])


In [27]:
# User prompt to run workflow
# user_prompt = "Run step 5, Rate articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("rate_articles")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


08:02:56 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Rating 86 AI articles using fn_rate_articles
08:02:56 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Calculating article ratings for 86 articles
08:02:56 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Rating recency
08:02:56 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Rating spam probability
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/rate_quality' from Langfuse
INFO:llm:Parsed prompt 'newsagent/rate_quality': model=gpt-4.1, system_len=1849, user_len=246


▶ Starting Step 5: step_05_rate_articles


08:03:00 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | low quality articles: {0.0: 62, 1.0: 13, 5.789356328730652e-12: 1, 0.00020342704214340812: 1, 4.691164021834418e-08: 1, 1.783247290814639e-11: 1, 1.493094676197164e-10: 1, 0.9626729941518225: 1, 0.002472623024627213: 1, 0.9820137760522224: 1, 6.82560337633487e-08: 1, 0.999998137537802: 1, 0.9988304590053244: 1}
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/rate_on_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/rate_on_topic': model=gpt-4.1, system_len=1790, user_len=240
08:03:02 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | on topic articles: {1.0: 53, 0.0: 4, 0.9999998063873693: 2, 1.1253517471925912e-07: 1, 0.004070137939983506: 1, 0.867035682761771: 1, 2.061153622438558e-09: 1, 9.736200313009565e-10: 1, 2.5690112979751028e-12: 1, 0.9947799085613173: 1, 0.9740425389805024: 1, 0.9984987679933719: 1, 1.783247290814639e-11: 1, 0.9999416367033446: 1, 

08:04:22 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | All ids: [41, 24, 42, 70, 30, 23, 31, 7, 46, 0, 21, 49, 28, 45, 38, 59, 64, 51, 29, 2, 56, 43, 18, 53, 60, 73, 1, 9, 33, 66, 50, 39, 25, 68, 14, 72, 19, 62, 48, 67, 10, 79, 5, 16, 76, 34, 22, 47, 82, 11, 54, 32, 74, 13, 83, 8, 63, 57, 15, 80, 3, 40, 55, 4, 81, 26, 69, 17, 65, 6, 12, 84, 61, 75, 52, 27, 58, 85, 36, 44, 20, 77, 35, 37, 78, 71]
08:04:22 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Processing 15 battles of size 6 with concurrency = 1000
08:05:04 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Battles complete
08:05:04 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | total battles: 621
08:05:04 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Recomputing Bradley-Terry ratings
08:05:04 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Recomputed Bradley-Terry ratings
08:05:04 | NewsletterAgent.test_newsletter_2025100608014325

08:07:17 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Generated 41 battle pairs
08:07:17 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | len(all_ids): 86 ; len(bt_df): 86
08:07:17 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Creating battle order
08:07:17 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | All ids: [24, 18, 41, 2, 46, 70, 23, 29, 28, 1, 30, 38, 42, 56, 45, 49, 21, 43, 31, 59, 0, 33, 7, 50, 51, 14, 60, 22, 53, 25, 64, 39, 9, 34, 19, 73, 48, 16, 68, 11, 10, 5, 67, 3, 62, 66, 47, 55, 32, 8, 54, 52, 72, 63, 4, 79, 74, 40, 76, 27, 57, 13, 6, 61, 26, 44, 69, 20, 12, 58, 65, 15, 82, 81, 80, 17, 83, 77, 84, 36, 35, 85, 75, 37, 78, 71]
08:07:17 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Processing 15 battles of size 6 with concurrency = 1000
08:07:53 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Battles complete
08:07:53 | NewsletterAgent.test_newsletter_20251006080143257231

⏱️  Total execution time: 331.36s
📊 Final result:
✅ Step 5 step_05_rate_articles completed successfully! Rated 83 articles with average rating 6.0/10.
⭐ High quality articles (≥7.0): 32
💾 Ratings stored in persistent state.


In [28]:
# step 7
# 0. get existing hdb model and apply
# 1. take clusters
# 2. extract topics prompt
# 3. combine with clusters and clean topics
# 4. add other
# 5. loop through articles and select best topic or other

# step 8 write inidividual sections
# step 9 put it all together


In [29]:
headline_df=state.headline_df
headline_df.sort_values("rating", ascending=False)


,source,title,url,published,rss_summary,id,isAI,status,final_url,html_path,...,input_text,age,recency_score,low_quality,on_topic,important,bt_z,adjusted_len,rating,bradley_terry
0,FT,OpenAI targets 10% AMD stake via multibillion-...,https://www.ft.com/content/bfafd06e-0a92-4add-...,NaN,NaN,0,True,exists,https://www.ft.com/content/bfafd06e-0a92-4add-...,download/html/OpenAI_targets_10__AMD_stake_via...,...,OpenAI targets 10% AMD stake via multibillion-...,1.000002,-0.000001,0.000000,1.000000e+00,1.0,3.862166,0.126456,10.988620,9.144806
1,FT,AstraZeneca signs $555mn AI deal to develop ge...,https://www.ft.com/content/c4b5153f-be07-454d-...,NaN,NaN,1,True,exists,https://www.ft.com/content/c4b5153f-be07-454d-...,download/html/AstraZeneca_signs__555mn_AI_deal...,...,AstraZeneca signs $555mn AI deal to develop ge...,1.000002,-0.000001,0.000000,1.000000e+00,1.0,3.859454,0.124178,10.983630,9.129421
2,FT,European private capital firms target €17bn in...,https://www.ft.com/content/48a39d2c-f5ac-470e-...,NaN,NaN,2,True,exists,https://www.ft.com/content/48a39d2c-f5ac-470e-...,download/html/European_private_capital_firms_t...,...,European private capital firms target €17bn in...,1.000002,-0.000001,0.000000,1.000000e+00,1.0,3.841896,0.118265,10.960159,9.029829
24,Feedly AI,AMD and OpenAI announce strategic partnership ...,https://openai.com/index/openai-amd-strategic-...,NaN,NaN,24,True,exists,https://openai.com/index/openai-amd-strategic-...,download/html/AMD_and_OpenAI_announce_strategi...,...,AMD and OpenAI announce strategic partnership ...,1.000002,-0.000001,0.000000,1.000000e+00,1.0,5.133062,0.583426,9.716486,16.353804
7,Reddit,"AI models tend to flatter users, and that prai...",https://www.theregister.com/2025/10/05/ai_mode...,NaN,NaN,7,True,exists,https://www.theregister.com/2025/10/05/ai_mode...,download/html/AI_models_tend_to_flatter_users_...,...,"AI models tend to flatter users, and that prai...",1.000002,-0.000001,0.000000,1.000000e+00,1.0,3.653484,0.757927,9.411410,7.961087
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,Feedly AI,AI won’t just eliminate millions of jobs. It w...,https://www.fastcompany.com/91405505/ai-wont-j...,NaN,NaN,77,True,None,https://www.fastcompany.com/91405505/ai-wont-j...,,...,AI won’t just eliminate millions of jobs. It w...,1.000002,-0.000001,1.000000,2.061154e-09,0.0,0.133235,0.000000,1.133234,-12.007079
78,Feedly AI,The right way to use AI at work,https://www.fastcompany.com/91413470/right-way...,NaN,NaN,78,True,None,https://www.fastcompany.com/91413470/right-way...,,...,The right way to use AI at work\n- no content,1.000002,-0.000001,1.000000,0.000000e+00,0.0,-0.078276,0.000000,0.921723,-13.206848
75,Reddit,Puppies + WWE with Sora 2,https://v.redd.it/rmxif4wk4dtf1,NaN,NaN,75,True,exists,https://v.redd.it/rmxif4wk4dtf1,download/html/Puppies___WWE_with_Sora_2.html,...,Puppies + WWE with Sora 2\n- The article discu...,1.000002,-0.000001,0.002473,9.984988e-01,0.0,-0.225455,0.095866,0.866436,-14.041706
81,NewsAPI,How AI Interprets Paintings by Van Gogh,https://www.kuriositas.com/2025/10/how-ai-inte...,2025-10-05T10:22:00Z,NaN,81,True,exists,https://www.kuriositas.com/2025/10/how-ai-inte...,download/html/How_AI_Interprets_Paintings_by_V...,...,How AI Interprets Paintings by Van Gogh\n- AI ...,1.000002,-0.000001,0.000000,3.466327e-07,0.0,0.503903,0.290702,0.794604,-9.904517


In [31]:
# User prompt to run workflow
# user_prompt = "Run step 6, Cluster articles by topic"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("cluster_by_topic")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


08:18:34 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Starting topic extraction for clustering
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/extract_topics' from Langfuse
INFO:llm:Parsed prompt 'newsagent/extract_topics': model=gpt-4.1-mini, system_len=1100, user_len=80
08:18:34 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Using model 'gpt-4.1-mini' for topic extraction
08:18:34 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Processing 83 articles for topic extraction


▶ Starting Step 6: step_06_cluster_by_topic


08:18:39 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Successfully extracted 322 total topics across articles
08:18:39 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Starting canonical topic classification for 101 topics
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/can

INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized Langfu

INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_top

...

=== HDBSCAN Parameters ===
min_cluster_size:   3
min_samples:        2
n_components:       366
=== Clustering Quality Metrics ===
Number of clusters: 6
Noise points: 55 (66.3%)
Average cluster size: 4.7 ± 1.2
Cluster size range: 3 - 7
=== Quality Scores ===
Silhouette Score: 0.154 (higher is better)
Calinski-Harabasz Score: 3.9 (higher is better)
Davies-Bouldin Score: 1.681 (lower is better)
HDBSCAN Validity Index: 0.060
Composite Score: 0.107 (higher is better)

[I 2025-10-06 08:19:26,466] Trial 191 finished with value: -0.10714036668114797 and parameters: {'n_components': 366, 'min_cluster_size': 3, 'min_samples': 2}. Best is trial 155 with value: -0.2438102853584599.
=== HDBSCAN Parameters ===
min_cluster_size:   3
min_samples:        2
n_components:       30
=== Clustering Quality Metrics ===
Number of clusters: 7
Noise points: 45 (54.2%)
Average cluster size: 5.4 ± 4.0
Cluster size range: 3 - 15
=== Quality Scores ===
Silhouette Score: 0.260 (higher is better)
Calinski-Harabasz Sc

/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/topic_writer' from Langfuse
INFO:llm:Parsed prompt 'newsagent/topic_writer': model=gpt-4.1, system_len=377, user_len=57


Reduced dimensions from 3072 to 24
=== Clustering Quality Metrics ===
Number of clusters: 8
Noise points: 46 (55.4%)
Average cluster size: 4.6 ± 1.9
Cluster size range: 3 - 9
=== Quality Scores ===
Silhouette Score: 0.238 (higher is better)
Calinski-Harabasz Score: 8.0 (higher is better)
Davies-Bouldin Score: 1.251 (lower is better)
HDBSCAN Validity Index: 0.067
Composite Score: 0.153 (higher is better)




08:19:27 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | 4: AI-Driven Investments in Advanced Hardware
08:19:27 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | OpenAI targets 10% AMD stake via multibillion-dollar chip deal (OpenAI, AMD, Semiconductors, AI Infrastructure, Mergers and Acquisitions, Chip Deals, Hardware)
America is now one big bet on AI (Finance, US Economy, AI Investment, Gen AI, Economics)
2025 Climate Tech Companies to Watch: Redwood Materials and its new AI microgrids (Redwood Materials, Battery Recycling, Renewable Energy, AI Microgrids, Supply chain optimization, Energy, Infrastructure)
AMD and OpenAI announce strategic partnership to deploy 6 gigawatts of AMD GPUs (OpenAI, AMD, GPU Deployment, AI Infrastructure, Hardware, Finance, Artificial General Intelligence)
OpenAI just gave AMD a big boost in the AI chip wars (OpenAI, AMD, AI Chips, Hardware, Deals, Finance, Infrastructure)
Source: xAI is set to spend $18B+ to acquire ~300K mor

⏱️  Total execution time: 58.25s
📊 Final result:
✅ Step 6 step_06_cluster_by_topic completed successfully! Organized 83 articles into topic clusters.


In [10]:
# User prompt to run workflow
# user_prompt = "Run step 7, select section topics"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("select_sections")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

12:18:31 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Free form categorization of articles
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/cat_proposal' from Langfuse
INFO:llm:Parsed prompt 'newsagent/cat_proposal': model=gpt-5-mini, system_len=638, user_len=1179
12:18:31 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Initialized LLMagent:
system_prompt: # Role & Objective
You are **“The News Pulse Analyst.”**
Your task: read a daily batch of AI-related news items and surface ** 10-30 ** short, high-impact topic titles for an executive summary.
You will receive today's AI-related news items in markdown format.
Each item will have headline, URL, topics, an item rating, and bullet-point summary.
Return ** 10-30 ** distinct, high-impact topics in the supplied JSON format.
Ensure that you propose topics that cover most of the highest-rated items (rated 7 and above)

# Input Format
Headline - Site

Rating: x.x

Topics

▶ Starting Step 7: step_07_select_sections


12:19:08 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Result: items=['OpenAI AMD GPU Partnership', 'OpenAI Stargate Data Centers', 'AI VC Funding Surge', 'xAI Nvidia Chip Expansion', 'GitHub Issue Agent Hijack', 'FieldAI Robotics Data Flywheel', 'Hybrid RAG Hallucination Fix', 'AI Models Sycophancy Study', 'European Data Center Sales', 'Small Modular Reactors Debate', 'Redwood AI Battery Microgrids', 'California AI Safety Law', 'ChatGPT Instant Checkout', 'Perplexity Comet Productivity Browser', 'Copilot Workplace Effects', 'Anthropic Context Engineering', 'RidgeGen Autonomous Security', 'Sora Likeness Controls', 'OpenAI Palm Device', 'Data Realism Debate']
12:19:08 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Cleaning up initial categories: ['AI Industry Economic Risks and Monopolies', 'European Data Center Sales', 'OpenAI AMD GPU Partnership', 'Copilot Workplace Effects', 'Perplexity Comet Productivity Browser', 'RidgeGen Autonomous Security', 'O

12:19:38 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | User message: CANDIDATE TOPICS
AI Hardware Investments
AI Market Concentration
AI Societal Impacts
AI Sycophancy Study
AI VC Funding Surge
AI Video Ethics
Anthropic Context Engineering
California AI Safety Law
ChatGPT Instant Checkout
Data Realism Debate
European Data Center Investments
FieldAI Robotics Data Flywheel
GitHub Issue Agent Hijack
Hybrid RAG Hallucination Fix
Microsoft Copilot Effects
OpenAI AMD Partnership
OpenAI Palm Device
OpenAI Stargate Data Centers
Perplexity Comet
Redwood AI Battery Microgrids
RidgeGen Autonomous Security
Small Modular Reactors Debate
Sora Likeness Controls
xAI Nvidia Chip Expansion

Classify the news item into exactly one of the candidate topics above. If your best match is < 60% confidence, output Other.

NEWS ITEM
OpenAI targets 10% AMD stake via multibillion-dollar chip deal - Financial Times

Rating: 11.0

Topics: OpenAI, AMD, Semiconductors, AI Infrastructure, Mergers and 

12:19:38 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | User message: CANDIDATE TOPICS
AI Hardware Investments
AI Market Concentration
AI Societal Impacts
AI Sycophancy Study
AI VC Funding Surge
AI Video Ethics
Anthropic Context Engineering
California AI Safety Law
ChatGPT Instant Checkout
Data Realism Debate
European Data Center Investments
FieldAI Robotics Data Flywheel
GitHub Issue Agent Hijack
Hybrid RAG Hallucination Fix
Microsoft Copilot Effects
OpenAI AMD Partnership
OpenAI Palm Device
OpenAI Stargate Data Centers
Perplexity Comet
Redwood AI Battery Microgrids
RidgeGen Autonomous Security
Small Modular Reactors Debate
Sora Likeness Controls
xAI Nvidia Chip Expansion

Classify the news item into exactly one of the candidate topics above. If your best match is < 60% confidence, output Other.

NEWS ITEM
OpenAI Agrees to Use Computer Chips from AMD - The New York Times

Rating: 7.3

Weeks after a $100 billion agreement to use computer chips from Nvidia, OpenAI has s

...

12:19:54 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | User message: Deduplicate the following news articles:
{"id":{"10":10,"48":48,"27":27,"62":62,"75":75},"extended_summary":{"10":"Sora provides better control over videos featuring your AI self\n\n\ufeffIt won\u2019t help stop the avalanche of AI slop OpenAI has unleashed.\n\nOpenAI, Deepfakes, Content Moderation, Ethics, Privacy, Misinformation, Policy And Regulation\n\n- Sora, an app by OpenAI described as a 'TikTok for deepfakes,' now allows users to limit how their AI-generated doubles can be used, including restricting appearances in political videos or usage of certain words.\n- These updates aim to give users greater control over AI deepfake content featuring themselves amidst concerns of misinformation and content misuse on the platform.\n- OpenAI acknowledges ongoing challenges with content moderation and plans to enhance safety features and restrictions to better protect users from misuse of AI-generated 

12:19:54 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | User message: Deduplicate the following news articles:
{"id":{"15":15,"59":59,"76":76},"extended_summary":{"15":"Elon Musk Gambles Billions in Memphis to Catch Up on AI\n\n- no content","59":"This AI skeptic got feedback from the industry \u2014 and now he\u2019s even more pessimistic\n\nWhat happens when an AI skeptic gets feedback from people in the industry? He gets even more bearish that the torrent of spending can be justified.\n\nData Centers, Finance, Economic Risk, Investment Cycle, Hardware, Nvidia, Venture Capital\n\n- Hedge fund founder Harris Kupperman revises his estimate, concluding AI data centers depreciate in 3 to 10 years instead of 10, drastically increasing the required revenue to justify the massive $30 billion monthly data-center spending.\n- Kupperman warns of a significant economic risk due to the rapid obsolescence of AI infrastructure, comparing the AI buildout to historic capital-heavy p

12:19:54 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | User message: Deduplicate the following news articles:
{"id":{"9":9,"69":69},"extended_summary":{"9":"Stop Hallucinations at the Source: Hybrid RAG That Checks Itself\n\nStop hallucinations. Validate every answer. Combine vector and graph search.\n\nDocuChat, Retrieval Augmented Generation, AI Hallucinations, Privacy, Finance, Healthcare, Knowledge Graphs\n\n- DocuChat is an open-source, local-first Retrieval-Augmented Generation (RAG) system that combines hybrid vector and knowledge graph search with self-critique to prevent AI hallucinations and validate responses.\n- The system utilizes parallel retrieval (vector and graph) with cross-modal validation, semantic grounding via sentence-level embeddings, adaptive confidence thresholds, and persona-driven response customization to meet diverse domain needs in fields such as finance, legal, and healthcare.\n- Designed for scalability from local laptops to enterprise

12:19:54 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | User message: Deduplicate the following news articles:
{"id":{"71":71,"81":81},"extended_summary":{"71":"How to Create a Homeless AI Tiktok Video & TikTok\u2019s Homeless AI Trend Explained\n\nDiscover how the Homeless AI trend on TikTok is using AI art to humanize homelessness and inspire meaningful conversations.\n\nAI Art, Ethics, Privacy, Content Creation, Homelessness, Society And Culture, TikTok Trends\n\n- The \"Homeless AI\" TikTok trend uses AI-generated visuals from tools like Midjourney and DALL-E to create emotionally impactful narratives focusing on homelessness, blending creativity with social awareness.\n- The content creation involves selecting high-quality, privacy-conscious base images, generating and customizing lifelike characters, sequencing images to build a compelling story arc, and enhancing realism through filters and watermark removal.\n- Ethical considerations are emphasized to protect p

✅ Completed Step 7: Categories and article counts:
cat
AI Hardware Investments              3
AI Market Concentration              3
AI Societal Impacts                 18
AI VC Funding Surge                  7
AI Video Ethics                      2
Anthropic Context Engineering        2
Data Realism Debate                  3
European Data Center Investments     2
FieldAI Robotics Data Flywheel       3
Hybrid RAG Hallucination Fix         2
OpenAI AMD Partnership               1
OpenAI Palm Device                   1
OpenAI Stargate Data Centers         1
Other                                8
RidgeGen Autonomous Security         2
Sora Likeness Controls               4
⏱️  Total execution time: 100.39s
📊 Final result:
Categories and article counts:
cat
AI Hardware Investments              3
AI Market Concentration              3
AI Societal Impacts                 18
AI VC Funding Surge                  7
AI Video Ethics                      2
Anthropic Context Engineering        2
Da

In [11]:
state.get_completed_steps() 



['step_01_fetch_urls',
 'step_02_filter_urls',
 'step_03_download_articles',
 'step_04_extract_summaries',
 'step_05_rate_articles',
 'step_06_cluster_by_topic',
 'step_07_select_sections']

In [12]:
# User prompt to run workflow
user_prompt = "Show the workflow status"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


📝 User prompt: 'Show the workflow status'


12:20:33 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Starting check_workflow_status
12:20:33 | NewsletterAgent.test_newsletter_20251006080143257231 | INFO | Completed check_workflow_status


⏱️  Total execution time: 11.64s
📊 Final result:
Current workflow status:
- Progress: 77.8% (7/9 complete)
- Status: 7 complete, 0 started, 0 failed, 2 not started
- Next step: Step 8 — Draft Sections

Step details:
- Step 1: Fetch Urls — complete
- Step 2: Filter Urls — complete
- Step 3: Download Articles — complete
- Step 4: Extract Summaries — complete
- Step 5: Rate Articles — complete
- Step 6: Cluster By Topic — complete
- Step 7: Select Sections — complete
- Step 8: Draft Sections — not_started
- Step 9: Finalize Newsletter — not_started

Categories and article counts (from state):
- AI Societal Impacts: 18
- AI VC Funding Surge: 7
- Other: 8
- AI Hardware Investments: 3
- AI Market Concentration: 3
- FieldAI Robotics Data Flywheel: 3
- Data Realism Debate: 3
- Sora Likeness Controls: 4
- Hybrid RAG Hallucination Fix: 2
- Anthropic Context Engineering: 2
- European Data Center Investments: 2
- RidgeGen Autonomous Security: 2
- OpenAI AMD Partnership: 1
- OpenAI Palm Device: 1
-

TODO:

- write sections
- initial write - prompt and output json for each section asynchronously
- check and rewrite each section for format asynchronsously

- assemble sections
- do a critic loop